<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Ouvrir dans Colab
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/sangalo20/Serverless-agents-cloudrun/blob/main/serverless_agents.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) | [Sangalo Mwenyinyo](https://github.com/sangalo20) |

# Agents Serverless sur Cloud Run

Bienvenue à l'atelier **Agents Serverless** ! Dans cette session, nous irons au-delà des simples chatbots et construirons un **Système Multi-Agents** prêt pour la production et orienté événements.

## Architecture

**Le Défi :**
Créer une application GenAI est facile, mais en créer une qui soit **évolutive**, **à jour** et **réactive** est difficile. Comment s'assurer que notre agent connaît les dernières informations sans l'entraîner constamment ? Comment éviter que le temps de traitement des documents longs ne bloque le chat de l'utilisateur ?

**La Solution : Micro-Agents**
Nous allons résoudre ce problème en décomposant notre système en deux micro-services spécialisés et indépendants. Ce modèle de "Micro-Agent" nous permet de séparer le travail lourd de l'ingestion de données des demandes en temps réel de l'interaction utilisateur.

Nous allons construire :
1.  **Le Bibliothécaire (Agent d'Ingestion)** : Un service d'arrière-plan orienté événements. Son seul travail est d'écouter les nouvelles informations (PDFs téléchargés sur Cloud Storage), de les lire, de les comprendre à l'aide de Gemini, et de les classer dans notre base de connaissances (Firestore). Il passe à zéro lorsqu'il n'est pas utilisé et s'adapte instantanément lorsque vous téléchargez des milliers de fichiers.
2.  **Le Guide (Agent d'Interface)** : Un service de chat destiné à l'utilisateur. Il se connecte à la base de connaissances créée par le Bibliothécaire pour répondre aux questions des utilisateurs avec précision. Il conserve l'historique des conversations et fournit une interface utile et humaine.

## Technologies Utilisées

> **[Google Cloud Run](https://cloud.google.com/run)**
> Cloud Run est une plateforme de calcul entièrement gérée qui vous permet d'exécuter des conteneurs directement sur l'infrastructure évolutive de Google. Elle fait abstraction de la gestion de l'infrastructure, vous permettant de vous concentrer sur la construction de vos agents. Elle s'adapte automatiquement de zéro à l'infini, ce qui signifie que vous ne payez que lorsque votre code est en cours d'exécution.

> **[Vertex AI & Gemini 2.5 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-models)**
> Gemini 2.5 Flash est le dernier modèle multimodal léger et à faible latence de Google, conçu pour les tâches à haute fréquence. Il offre une vitesse et une rentabilité exceptionnelles tout en maintenant des capacités de raisonnement élevées.

> **[Eventarc](https://cloud.google.com/eventarc)**
> Eventarc vous permet de construire des architectures orientées événements en acheminant les événements des sources Google Cloud (comme Cloud Storage) vers vos services. Nous utilisons Eventarc pour déclencher notre agent "Bibliothécaire" instantanément à chaque téléchargement de fichier.

> **[Firestore](https://cloud.google.com/firestore)**
> Firestore est une base de données cloud NoSQL flexible et évolutive pour stocker et synchroniser les données. Nous utilisons Firestore comme le "Cerveau" de notre système.

C'est parti !


## 1. Configuration et Authentification

**Pourquoi avons-nous besoin de cela ?**
Pour interagir avec les ressources Google Cloud (comme Cloud Run, Firestore, etc.) depuis ce notebook, nous devons prouver qui nous sommes.

**Qu'est-ce que l'ADC (Application Default Credentials) ?**
ADC est une stratégie utilisée par les bibliothèques Google Cloud pour trouver automatiquement vos identifiants. En exécutant `gcloud auth login --update-adc`, nous plaçons vos identifiants personnels dans un emplacement connu.

Si vous n'avez pas encore de projet :

1. [Créez un projet](https://console.cloud.google.com/projectcreate) dans la console Google Cloud.
2. Copiez votre `ID de projet` depuis la page [Paramètres](https://console.cloud.google.com/iam-admin/settings).


In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
REGION = "us-central1"  # @param {type:"string", isTemplate: true}

if PROJECT_ID == "[your-project-id]" or not PROJECT_ID:
    print("Please specify your project id in PROJECT_ID variable.")
    raise KeyboardInterrupt

!gcloud auth print-identity-token -q &> /dev/null || gcloud auth login --project="{PROJECT_ID}" --update-adc --quiet

!gcloud config set project {PROJECT_ID}
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = REGION

## 1.1 Cloner le dépôt

**Pourquoi faisons-nous cela ?**
Google Colab est une machine virtuelle temporaire. Elle démarre vide. Le code de nos agents (`main.py`, `Dockerfile`) se trouve sur GitHub. Nous devons cloner (`git clone`) ce code dans cette machine pour pouvoir le construire et le déployer.


In [ ]:
!git clone https://github.com/sangalo20/Serverless-agents-cloudrun.git
%cd Serverless-agents-cloudrun

## 1.2 Définir les Dockerfiles

**Infrastructure as Code**
Pour avoir un contrôle total, nous définirons nos Dockerfiles ici même dans le notebook. Cela nous permet de voir exactement comment notre conteneur est construit.


In [ ]:
%%writefile librarian/Dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# Run with Gunicorn
CMD ["gunicorn", "--bind", "0.0.0.0:8080", "--workers", "1", "--threads", "8", "--timeout", "0", "main:app"]

In [ ]:
%%writefile guide/Dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

# Run with Gunicorn
CMD ["gunicorn", "--bind", "0.0.0.0:8080", "--workers", "1", "--threads", "8", "--timeout", "0", "main:app"]

## 2. Activer les API

**Qu'est-ce que c'est ?**
Les services Google Cloud ne sont pas activés par défaut. Nous devons activer les services spécifiques que nous prévoyons d'utiliser :
*   `run.googleapis.com`: **Cloud Run** (pour exécuter nos conteneurs).
*   `eventarc.googleapis.com`: **Eventarc** (pour déclencher le Bibliothécaire).
*   `aiplatform.googleapis.com`: **Vertex AI** (pour utiliser le modèle Gemini).
*   `firestore.googleapis.com`: **Firestore** (notre base de données).
*   `cloudbuild.googleapis.com`: **Cloud Build** (pour construire nos conteneurs).
*   `storage.googleapis.com`: **Cloud Storage** (pour stocker les fichiers PDF).
*   `artifactregistry.googleapis.com`: **Artifact Registry** (pour stocker nos images Docker).


In [ ]:
!gcloud services enable run.googleapis.com eventarc.googleapis.com aiplatform.googleapis.com firestore.googleapis.com cloudbuild.googleapis.com storage.googleapis.com artifactregistry.googleapis.com

## 3. Créer l'Infrastructure

**Le Plan :**
1.  **Bucket Cloud Storage** : Nous avons besoin d'un endroit pour télécharger nos fichiers PDF. Ce bucket agira comme la "Boîte de réception" pour notre agent Bibliothécaire.
2.  **Permissions** : Nous nous assurons que notre compte de service de build a la permission d'écrire des logs, de sauvegarder des images, d'accéder à Vertex AI et d'écrire dans Firestore.
3.  **Base de données Firestore** : Nous avons besoin d'une base de données rapide et serverless pour stocker les *connaissances résumées* et l'*historique du chat*.
4.  **Artifact Registry** : Nous avons besoin d'un dépôt pour stocker nos images Docker.


In [ ]:
BUCKET_NAME = f"{PROJECT_ID}-knowledge-base"
!gsutil mb -l {REGION} gs://{BUCKET_NAME}
print(f"Created bucket: {BUCKET_NAME}")

# Create Firestore in Native mode (if not exists)
!gcloud firestore databases create --location={REGION} --type=firestore-native

# Create Artifact Registry Repository
!gcloud artifacts repositories create containers --repository-format=docker --location={REGION} --description="Docker repository"

## 3.1 Configurer les Permissions

**Accorder l'accès**
Nous devons nous assurer que notre compte de service de build a la permission d'écrire des logs, de sauvegarder des images, d'accéder à Vertex AI et d'écrire dans Firestore.


In [ ]:
# Get Project Number and Service Account
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
SERVICE_ACCOUNT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

# Grant permissions for Cloud Build and Runtime (Logging, Artifact Registry, Vertex AI, Firestore)
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/logging.logWriter
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/artifactregistry.writer
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/storage.admin
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/aiplatform.user
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/datastore.user

# Grant Pub/Sub Publisher to Cloud Storage Service Agent (required for Eventarc)
# 1. Force create the Service Agent (if it doesn't exist)
!gcloud beta services identity create --service=storage.googleapis.com --project={PROJECT_ID}

# 2. Construct the email manually (now that we know it exists)
GCS_SERVICE_AGENT = f"service-{PROJECT_NUMBER}@gs-project-accounts.iam.gserviceaccount.com"

print(f"GCS Service Agent: {GCS_SERVICE_AGENT}")
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{GCS_SERVICE_AGENT} --role=roles/pubsub.publisher

## 4. Construire le service "Le Bibliothécaire"

**Étape 1 : Construire le Conteneur**
Nous utilisons `gcloud builds submit` pour empaqueter notre code python (`librarian/main.py`) dans une image de conteneur Docker. Cette image est stockée dans Artifact Registry.


In [ ]:
SERVICE_NAME_LIBRARIAN = "librarian"
!gcloud builds submit --tag {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_LIBRARIAN} librarian/

## 4.1 Déployer le service "Le Bibliothécaire"

**Étape 2 : Déployer sur Cloud Run**
Maintenant, nous prenons l'image que nous venons de construire et la déployons sur Cloud Run. Nous utilisons `--allow-unauthenticated` pour qu'Eventarc puisse facilement le déclencher.


In [ ]:
!gcloud run deploy {SERVICE_NAME_LIBRARIAN} --image {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_LIBRARIAN} --region {REGION} --allow-unauthenticated

## 5. Construire le service "Le Guide"

**Étape 1 : Construire le Conteneur**
Similaire au Bibliothécaire, nous construisons d'abord l'image du conteneur pour le service Guide.


In [ ]:
SERVICE_NAME_GUIDE = "guide"
!gcloud builds submit --tag {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_GUIDE} guide/

## 5.1 Déployer le service "Le Guide"

**Étape 2 : Déployer sur Cloud Run**
Nous déployons le service Guide. Ce service hébergera le point de terminaison du chat.


In [ ]:
!gcloud run deploy {SERVICE_NAME_GUIDE} --image {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_GUIDE} --region {REGION} --allow-unauthenticated

## 6. Connecter le tout avec Eventarc

**La Colle Magique**
Pour l'instant, le service Bibliothécaire fonctionne, mais il ne sait pas quand un fichier est téléchargé. Nous avons besoin d'**Eventarc** pour faire le lien.

Nous créons un **Déclencheur** qui dit :
*   **SI** un fichier est `finalisé` (téléchargé) ...
*   **DANS** le bucket spécifique `{BUCKET_NAME}` ...
*   **ALORS** envoyer une requête POST au service `{SERVICE_NAME_LIBRARIAN}`.

*Note : Nous accordons également les permissions IAM nécessaires pour qu'Eventarc soit autorisé à appeler notre service Cloud Run.*


In [ ]:
# Grant permission to the Compute Engine service account (default for Eventarc)
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
SERVICE_ACCOUNT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/eventarc.eventReceiver
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/run.invoker

# Create the trigger
!gcloud eventarc triggers create librarian-trigger \
  --location={REGION} \
  --destination-run-service={SERVICE_NAME_LIBRARIAN} \
  --destination-run-region={REGION} \
  --event-filters="type=google.cloud.storage.object.v1.finalized" \
  --event-filters="bucket={BUCKET_NAME}" \
  --service-account={SERVICE_ACCOUNT}

## 7. Testez-le !

**Interface Utilisateur Interactive**
Testons nos agents avec une vraie interface utilisateur ! Utilisez les boutons ci-dessous pour télécharger un PDF ou un fichier texte, puis discutez avec votre agent.


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import requests
import os
from google.cloud import storage

# --- Configuration ---
# Get Guide URL dynamically
try:
    GUIDE_URL = !gcloud run services describe {SERVICE_NAME_GUIDE} --region {REGION} --format='value(status.url)'
    GUIDE_URL = GUIDE_URL[0]
except:
    print("Error: Could not find Guide service URL. Make sure it is deployed.")
    GUIDE_URL = ""

SESSION_ID = "interactive-session"

# --- UI Elements ---

# 1. Upload Section
upload_btn = widgets.Button(description="Télécharger un document", button_style='info', icon='upload')
upload_out = widgets.Output()

def on_upload_clicked(b):
    with upload_out:
        upload_out.clear_output()
        print("Sélectionnez un fichier à télécharger...")
        uploaded = files.upload()
        
        # Initialize Storage Client
        try:
            storage_client = storage.Client()
            bucket = storage_client.bucket(BUCKET_NAME)
            
            for filename in uploaded.keys():
                print(f"Téléchargement de {filename} vers {BUCKET_NAME}...")
                try:
                    blob = bucket.blob(filename)
                    blob.upload_from_filename(filename)
                    print(f"✅ {filename} téléchargé avec succès ! Le Bibliothécaire le traite (attendez ~10-20s)...")
                except Exception as e:
                    print(f"❌ Erreur lors du téléchargement de {filename} : {e}")
        except Exception as e:
             print(f"❌ Erreur lors de l'initialisation du client de stockage : {e}")

upload_btn.on_click(on_upload_clicked)

# 2. Chat Section
chat_history = widgets.Output(layout={'border': '1px solid #ccc', 'height': '300px', 'overflow_y': 'scroll'})
user_input = widgets.Text(placeholder='Posez une question sur votre document...', layout={'width': '70%'}) 
send_btn = widgets.Button(description="Envoyer", button_style='primary')

def on_send_clicked(b):
    question = user_input.value
    if not question: return
    
    # Display User Message
    with chat_history:
        print(f"Vous : {question}")
    
    user_input.value = '' # Clear input
    
    # Call API
    try:
        response = requests.post(f"{GUIDE_URL}/chat", json={"session_id": SESSION_ID, "query": question})
        if response.status_code == 200:
            data = response.json()
            answer = data.get('answer', 'No answer received.')
            with chat_history:
                print(f"Agent : {answer}\n")
        else:
            with chat_history:
                print(f"Error: {response.status_code} - {response.text}\n")
    except Exception as e:
        with chat_history:
            print(f"Connection Error: {e}\n")

send_btn.on_click(on_send_clicked)
user_input.on_submit(on_send_clicked)

# Layout
print(f"Connected to Guide Agent at: {GUIDE_URL}")
display(widgets.VBox([
    widgets.HTML("<h3>1. Base de Connaissances</h3>"),
    upload_btn,
    upload_out,
    widgets.HTML("&lt;hr&gt;<h3>2. Interface de Chat</h3>"),
    chat_history,
    widgets.HBox([user_input, send_btn])
]))


## 8. Nettoyage

**Nettoyez vos ressources**
Pour éviter des frais, supprimez les ressources utilisées dans cet atelier.


In [ ]:
print("Starting cleanup...\n")

# 1. Delete Eventarc Trigger
print(f"Deleting Eventarc trigger: librarian-trigger...")
!gcloud eventarc triggers delete librarian-trigger --location={REGION} --quiet
print("✅ Eventarc trigger deleted.\n")

# 2. Delete Cloud Run Services
print(f"Deleting Cloud Run service: {SERVICE_NAME_LIBRARIAN}...")
!gcloud run services delete {SERVICE_NAME_LIBRARIAN} --region {REGION} --quiet
print(f"✅ Service {SERVICE_NAME_LIBRARIAN} deleted.\n")

print(f"Deleting Cloud Run service: {SERVICE_NAME_GUIDE}...")
!gcloud run services delete {SERVICE_NAME_GUIDE} --region {REGION} --quiet
print(f"✅ Service {SERVICE_NAME_GUIDE} deleted.\n")

# 3. Delete Cloud Storage Bucket
print(f"Deleting Bucket: {BUCKET_NAME}...")
!gsutil rm -r gs://{BUCKET_NAME}
print(f"✅ Bucket {BUCKET_NAME} deleted.\n")

# 4. Delete Artifact Registry Repository
print("Deleting Artifact Registry repository: containers...")
!gcloud artifacts repositories delete containers --location={REGION} --quiet
print("✅ Artifact Registry repository deleted.\n")

# 5. Delete Firestore Database
print("Deleting Firestore database: (default)...")
!gcloud firestore databases delete --database="(default)" --quiet
print("✅ Firestore database deleted.\n")

print("Cleanup complete!")